# Fast Multitask Lattice GP

In [1]:
import fastgps
import torch
import numpy as np

In [2]:
torch.set_default_dtype(torch.float64)

## True Function

In [3]:
def f_ackley(x, a=20, b=0.2, c=2*np.pi, scaling=32.768):
    # https://www.sfu.ca/~ssurjano/ackley.html
    assert x.ndim==2
    x = 2*scaling*x-scaling
    t1 = a*torch.exp(-b*torch.sqrt(torch.mean(x**2,1)))
    t2 = torch.exp(torch.mean(torch.cos(c*x),1))
    t3 = a+np.exp(1)
    y = -t1-t2+t3
    return y
f_low_fidelity = lambda x: f_ackley(x,c=0)
f_high_fidelity = lambda x: f_ackley(x)
f_cos = lambda x: torch.cos(2*np.pi*x).sum(1)
fs = [f_low_fidelity,f_high_fidelity,f_cos]
d = 1 # dimension
rng = torch.Generator().manual_seed(17)
x = torch.rand((2**7,d),generator=rng) # random testing locations
y = torch.vstack([f(x) for f in fs]) # true values at random testing locations
z = torch.rand((2**8,d),generator=rng) # other random locations at which to evaluate covariance
print("x.shape = %s"%str(tuple(x.shape)))
print("y.shape = %s"%str(tuple(y.shape)))
print("z.shape = %s"%str(tuple(z.shape)))

x.shape = (128, 1)
y.shape = (3, 128)
z.shape = (256, 1)


## Construct Fast GP

In [4]:
fgp = fastgps.FastGPLattice(d,seed_for_seq=7,num_tasks=len(fs))
x_next = fgp.get_x_next(n=[2**6,2**3,2**8])
y_next = [fs[i](x_next[i]) for i in range(fgp.num_tasks)]
fgp.add_y_next(y_next)
assert len(x_next)==len(y_next)
for i in range(len(x_next)):
    print("i = %d"%i)
    print("\tx_next[%d].shape = %s"%(i,str(tuple(x_next[i].shape))))
    print("\ty_next[%d].shape = %s"%(i,str(tuple(y_next[i].shape))))

i = 0
	x_next[0].shape = (64, 1)
	y_next[0].shape = (64,)
i = 1
	x_next[1].shape = (8, 1)
	y_next[1].shape = (8,)
i = 2
	x_next[2].shape = (256, 1)
	y_next[2].shape = (256,)


In [5]:
pmean = fgp.post_mean(x)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("l2 relative error =",(torch.linalg.norm(y-pmean,dim=1)/torch.linalg.norm(y,dim=1)))

pmean.shape = (3, 128)
l2 relative error = tensor([0.0046, 0.0692, 0.0003])


In [6]:
data = fgp.fit()
list(data.keys())

     iter of 5.0e+03 | loss       | term1      | term2     
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            0.00e+00 | 4.82e+03   | 1.17e+04   | -2.67e+03 
            5.00e+00 | -3.62e+02  | 8.01e+02   | -2.13e+03 
            1.00e+01 | -6.69e+02  | 1.04e+02   | -2.04e+03 
            1.50e+01 | -1.26e+03  | 1.74e+02   | -3.30e+03 
            2.00e+01 | -1.56e+03  | 2.28e+02   | -3.94e+03 
            2.50e+01 | -1.57e+03  | 2.97e+02   | -4.04e+03 
            3.00e+01 | -1.57e+03  | 3.14e+02   | -4.06e+03 
            3.50e+01 | -1.57e+03  | 3.23e+02   | -4.07e+03 
            4.00e+01 | -1.57e+03  | 3.23e+02   | -4.07e+03 
            4.50e+01 | -1.57e+03  | 3.22e+02   | -4.07e+03 
            5.00e+01 | -1.57e+03  | 3.31e+02   | -4.08e+03 
            5.50e+01 | -1.58e+03  | 3.27e+02   | -4.09e+03 
            6.00e+01 | -1.58e+03  | 3.20e+02   | -4.09e+03 
            6.50e+01 | -1.59e+03  | 3.31e+02   | -4.11e+03 
            7.00e+01 | -1.59e+03  | 3.32

            8.00e+01 | -1.59e+03  | 3.25e+02   | -4.10e+03 
            8.50e+01 | -1.59e+03  | 3.26e+02   | -4.10e+03 
            9.00e+01 | -1.59e+03  | 3.36e+02   | -4.11e+03 
            9.50e+01 | -1.59e+03  | 3.25e+02   | -4.10e+03 
            1.00e+02 | -1.59e+03  | 3.26e+02   | -4.10e+03 
            1.05e+02 | -1.59e+03  | 3.25e+02   | -4.10e+03 
            1.10e+02 | -1.59e+03  | 3.24e+02   | -4.10e+03 
            1.15e+02 | -1.59e+03  | 3.24e+02   | -4.10e+03 
            1.20e+02 | -1.59e+03  | 3.22e+02   | -4.10e+03 
            1.25e+02 | -1.59e+03  | 3.25e+02   | -4.10e+03 
            1.30e+02 | -1.59e+03  | 3.25e+02   | -4.10e+03 
            1.35e+02 | -1.59e+03  | 3.27e+02   | -4.11e+03 
            1.40e+02 | -1.59e+03  | 3.24e+02   | -4.10e+03 
            1.45e+02 | -1.59e+03  | 3.22e+02   | -4.10e+03 
            1.50e+02 | -1.59e+03  | 3.25e+02   | -4.10e+03 
            1.55e+02 | -1.59e+03  | 3.25e+02   | -4.10e+03 
            1.60e+02 | -1.59e+03  | 3.24

            1.70e+02 | -1.59e+03  | 3.24e+02   | -4.10e+03 
            1.75e+02 | -1.59e+03  | 3.24e+02   | -4.10e+03 
            1.77e+02 | -1.59e+03  | 3.27e+02   | -4.11e+03 


['iterations']

In [7]:
pmean,pvar,q,ci_low,ci_high = fgp.post_ci(x,confidence=0.99)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("pvar.shape = %s"%str(tuple(pvar.shape)))
print("q = %.2f"%q)
print("ci_low.shape = %s"%str(tuple(ci_low.shape)))
print("ci_high.shape = %s"%str(tuple(ci_high.shape)))
print("l2 relative error =",(torch.linalg.norm(y-pmean,dim=1)/torch.linalg.norm(y,dim=1)))
pcov = fgp.post_cov(x,x)
print("pcov.shape = %s"%str(tuple(pcov.shape)))
_range0,_rangen1 = torch.arange(pcov.size(0)),torch.arange(pcov.size(-1))
assert torch.allclose(pcov[_range0,_range0][:,_rangen1,_rangen1],pvar) and (pvar>=0).all()
pcov2 = fgp.post_cov(x,z)
print("pcov2.shape = %s"%str(tuple(pcov2.shape)))

pmean.shape = (3, 128)
pvar.shape = (3, 128)
q = 2.58
ci_low.shape = (3, 128)
ci_high.shape = (3, 128)
l2 relative error = tensor([4.7055e-03, 5.7484e-02, 1.1823e-08])
pcov.shape = (3, 3, 128, 128)
pcov2.shape = (3, 3, 128, 256)


In [8]:
pcmean,pcvar,q,cci_low,cci_high = fgp.post_cubature_ci(confidence=0.99)
print("pcmean =",pcmean)
print("pcvar =",pcvar)
print("cci_low =",cci_low)
print("cci_high",cci_high)

pcmean = tensor([ 1.6943e+01,  1.8140e+01, -8.9762e-13])
pcvar = tensor([3.5905e-05, 2.9551e-03, 1.3137e-12])
cci_low = tensor([ 1.6928e+01,  1.8000e+01, -2.9523e-06])
cci_high tensor([1.6959e+01, 1.8280e+01, 2.9523e-06])


## Project and Increase Sample Size

In [9]:
n_new = fgp.n*torch.tensor([4,2,8])
pcov_future = fgp.post_cov(x,z,n=n_new)
pvar_future = fgp.post_var(x,n=n_new)
pcvar_future = fgp.post_cubature_var(n=n_new)

In [10]:
x_next = fgp.get_x_next(n_new)
y_next = [fs[i](x_next[i]) for i in range(fgp.num_tasks)]
for _y in y_next:
    print(_y.shape)
fgp.add_y_next(y_next)
print("l2 relative error =",(torch.linalg.norm(y-fgp.post_mean(x),dim=1)/torch.linalg.norm(y,dim=1)))
assert torch.allclose(fgp.post_cov(x,z),pcov_future)
assert torch.allclose(fgp.post_var(x),pvar_future)
assert torch.allclose(fgp.post_cubature_var(),pcvar_future)

torch.Size([192])
torch.Size([8])
torch.Size([1792])
l2 relative error = tensor([3.5162e-04, 5.9589e-02, 7.6261e-10])


In [11]:
data = fgp.fit(verbose=False)
print("l2 relative error =",(torch.linalg.norm(y-fgp.post_mean(x),dim=1)/torch.linalg.norm(y,dim=1)))

l2 relative error = tensor([3.4562e-04, 5.9561e-02, 5.8075e-12])


In [12]:
n_new = fgp.n*torch.tensor([4,8,2])
pcov_new = fgp.post_cov(x,z,n=n_new)
pvar_new = fgp.post_var(x,n=n_new)
pcvar_new = fgp.post_cubature_var(n=n_new)
x_next = fgp.get_x_next(n_new)
y_next = [fs[i](x_next[i]) for i in range(fgp.num_tasks)]
fgp.add_y_next(y_next)
assert torch.allclose(fgp.post_cov(x,z),pcov_new)
assert torch.allclose(fgp.post_var(x),pvar_new)
assert torch.allclose(fgp.post_cubature_var(),pcvar_new)